# **Lab3: Iterative Methods**
**Patrik Svensson**

# **Abstract**

In this lab report, the concept of iterative methods of solving linear equations, and also non-linear functions are explored. It is possible to solve linear equations by using direct method, but it is not always the most suitable from a computional and performance view, therefore iterative methods are presented here as a substitute. The report investigates four different algorithms for solving linear and non-linear equations, *Jacobi iteration*, *Gauss-Seidel method*, and *Newton's method for scalar nonlinear equation*. The result of the report is four implemented functions in Python, together with unit tests that confirms the correctness of the implementation of the algorithms.

# **Set up environment**

To set up the environment, run the two following lines of code.

In [0]:
import numpy as np
import unittest
import math as math
import scipy.misc

# **Introduction**

In the previous lab we solved linear equations on the classical form $Ax = b$ with direct methods, in other words, by doing factorization. But when the matrices get to big, on a million dimension level, the method by using factorization becomes unpractical. To mitigate this issue, we are introducing a new approach called *iterative methods*. Instead of finding an exact solution to a linear equation, we are satisfied with an aproximate solution. We achive this by iterating through an algorithm that is getting closer and closer to the solution of the linear eqation, and when it is close enough to the correct solution, the algorithm stops and return the solution (with a minor error). 


# **Methods**
In this chapter, I will present how the implementation of the functions was conducted. The study was conducted in the following way.

1.   Literature research
2.   Implementation
3.   Testing

In the sections below, I have provided a reference to where the algorithms were founded, or how it was deduced, followed with a code implementation in Python, and lastly unit test for the assurance of the accuracy of the implementations.


## Jacobi iteration
When deciding to solve a linear equation system with a Jacobi iteration, we start with the following expansion of the classic linear equation that we want to solve. 

$Ax = b \leftrightarrow
\\(D + L + U)x = b \leftrightarrow
\\Dx = b - (L + U)x \leftrightarrow
\\x = D^{-1}(b - (L + U)x)$

Where $D$ is the diagonal, $L$ is the lower triangular matrix, $U$ is the upper triangular matrix of $A$. The Jacobi iteration is a fixed-point iteration of the expansion above. When performing a Jacobi iteration, the algorithm is the following.

$x_0 = initial \: vector\\
x_{k+1} = D^{-1}(b - (L + U)x_k) \: \textbf{for} \: k = 0,1,2,3,...$

This according to (2.40) in the book *Numerical Analysis* that is a compiled version of Timonthy Sauer's book with the same made for KTH and published by Pearson. The code for the algorithm below is inpired by the matlab code in example 2.25 from the same book. The algorithm will continue to iterated until the residual is less than *TOL*. Below is an implementation of the algorithm in Python.


In [0]:
def jacobi_iteration(A, b):
  n = b.shape[0]
  diagonal = np.diag(np.diag(A))
  diagonal_inverted = np.linalg.inv(diagonal)
  r = A - np.diag(np.diag(A))
  x = np.zeros(n)
  TOL = 0.01
  residual = np.full(n, np.Infinity)
  might_diverge = False
  MAX_ITERATIONS = 100

  # Strictly diagonally dominant check
  diagonal_array = np.diag(A)
  for i in range(0, n):
    for j in range(0, n):
      if(A[i,j] > diagonal_array[i]):
        might_diverge = True

  if (might_diverge):
    for i in range(0, MAX_ITERATIONS):
      if not (np.linalg.norm(residual) > TOL):
        break
      residual = np.dot(A, x) - b 
      x = np.dot(diagonal_inverted,b-np.dot(r,x))
  else:
    while np.linalg.norm(residual) > TOL:
      residual = np.dot(A, x) - b 
      x = np.dot(diagonal_inverted,b-np.dot(r,x))

  if np.linalg.norm(residual) > TOL:
    return None

  return x


In the code below there is unit tests that are used to assure the correctness of the implemented Jacobi iteration.

In [0]:
class TestJacobiIteration(unittest.TestCase):
  def test_residual_Ax_b_one(self):
    # Arrange
    A = np.array([[3,1],
              [1,2]])
    expected_b = np.array([5,5])

    # Act
    x = jacobi_iteration(A, expected_b)
    b = np.dot(A, x)

    # Assert
    self.assertAlmostEqual(0, np.linalg.norm(expected_b - b), 2)

  def test_residual_x_y_one(self):
    # Arrange
    A = np.array([[3,1],
              [1,2]])
    b = np.array([5,5])
    expected_y = np.array([1,2])

    # Act
    x = jacobi_iteration(A, b)

    # Assert
    self.assertAlmostEqual(0, np.linalg.norm(x - expected_y), 2)

  def test_singular_matrix(self):
    # Arrange
    A = np.array([[1,2],
              [1,2]])
    b = np.array([5,5])
    # Act
    x = jacobi_iteration(A, b)

    # Assert
    self.assertEqual(None, x)

if __name__ == '__main__':
    # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
    # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


## Gauss-Seidel method
The Gauss-Seidel method aims to approximately solve and linear equation on the form $Ax = b$, and it is very similar to the Jacobi iteration, and can be seen as a tweak of the Jacobi iteration. The difference lies in the idea that as we are calculating the elements of $x$ vector sequentialy, we could use the already calculated elements in the calculation of the remaining elements as we calculate the remaining elements of the $x$ vector. From the standard form of the linear equation, we can expand it the following way.

$Ax = b
\\ (L + D + U)x = b
\\ (L + D)x = -Ux + b$

Which can be used as a fixed-point iteration:

$(L + D)x_{k+1} = -Ux_k + b
\\x_{k+1} = D^{-1}(b - Ux_k - Lx_{k+1})$

The Gauss-Seidel method can be expressed as an algorithm on the following form, which is taken from the book *Numerical Analysis* that is a compiled version of Timonthy Sauer's book with the same made for KTH and published by Pearson. The algorithm will continue to iterated until the residual is less than *TOL*.

$x_0 = initial \: vector\\
x_{k+1} = D^{-1}(b - Ux_k - Lx_{k+1}) \: \textbf{for} \: k = 0,1,2,3,...$

The code below is the implementation of Gauss-Seidel in Python.

In [0]:
def gauss_seidel_method(A, b):
  n = b.shape[0]
  diagonal = np.diag(np.diag(A))
  diagonal_inverted = np.linalg.inv(diagonal)
  L = np.tril(A) - diagonal
  U = np.triu(A) - diagonal
  x = np.zeros(n)
  TOL = 0.0001
  residual = np.full(n, np.Infinity)
  might_diverge = False
  MAX_ITERATIONS = 100

  # Strictly diagonally dominant check
  diagonal_array = np.diag(A)
  for i in range(0, n):
    for j in range(0, n):
      if(A[i,j] > diagonal_array[i]):
        might_diverge = True

  if (might_diverge):
    for i in range(0, MAX_ITERATIONS):
      if not (np.linalg.norm(residual) > TOL):
        break
      for i in range(0, n):
        x[i] = (b[i] - np.dot(U[i],x) - np.dot(L[i],x)) * diagonal_inverted[i][i]
      residual = np.dot(A, x) - b 
  else:
    while np.linalg.norm(residual) > TOL:
      residual = np.dot(A, x) - b 
      for i in range(0, n):
        x[i] = (b[i] - np.dot(U[i],x) - np.dot(L[i],x)) * diagonal_inverted[i][i]

  if np.linalg.norm(residual) > TOL:
    return None

  return x


In the code below there is unit tests that are used to assure the correctness of the implemented Gauss-Seidel method.

In [0]:
class TestGaussSeidelMethod(unittest.TestCase):
  def test_residual_Ax_b_one(self):
    # Arrange
    A = np.array([[3,1],
                  [1,2]])
    expected_b = np.array([5,5])

    # Act
    x = gauss_seidel_method(A, expected_b)
    b = np.dot(A, x)

    # Assert
    self.assertAlmostEqual(0, np.linalg.norm(expected_b - b), 2)

  def test_residual_x_y_one(self):
    # Arrange
    A = np.array([[3,1],
              [1,2]])
    b = np.array([5,5])
    expected_y = np.array([1,2])

    # Act
    x = gauss_seidel_method(A, b)

    # Assert
    self.assertAlmostEqual(0, np.linalg.norm(x - expected_y), 2)

  def test_singular_matrix(self):
    # Arrange
    A = np.array([[1,2],
                  [1,2]])
    b = np.array([10,5])
    # Act
    x = gauss_seidel_method(A, b)

    # Assert
    self.assertEqual(None, x)

if __name__ == '__main__':
    # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
    # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

## Newton's method for scalar nonlinear equation
Compared to the previous investigated algorithms, Jacobi iteration, and Gauss-Seidel method, they work well when we want to find a solution to a linear quation, but they do not work when we are trying to find the root to a non-linear function. Newton's method is an iterative algorithm, like the previously presented algorithms. The idea behind the algorithm is to getting closer and closer to the root by running the algorithm several time, in other words, iterating.

In the algorithm, you start by picking a arbitrary point $x_0$ in the domain of a $f(x)$ that you want to find to root in, which then you use to get the tangent-line of $f(x)$ in $x_0$. The value of x where the tangent-line cuts the x-axis is the new x value $x_1$ which is closer to the root if it converges. This procedure is repeated until the x value is sufficiently close to the root value. This process can be expressed on the following form:

$x_0 = initial \: guess
\\ x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)} \: \textbf{for} \: i = 0,1,2,...$

Which is provided in the book *Numerical Analysis* that is a compiled version of Timonthy Sauer's book with the same made for KTH and published by Pearson. The intial guess is a quite difficult issue, therefore I'm setting it to one.

The Python implementation below is based on the algorithm 8.1 from the course lecture notes. 

In [0]:
def newtons_method_scalar(f):
  x = 1
  TOL = 0.000001
  while np.linalg.norm(f(x)) > TOL:
    df = scipy.misc.derivative(f, x)
    x = x - f(x)/df
    
  return x


In the code below there is unit tests that are used to assure the correctness of the implemented Jacobi iteration.

In [0]:
def scalar_function_one(x):
  return x ** 3 + 1

def scalar_function_two(x):
  return (x - 3)**2

class TestNewtonMethodScalar(unittest.TestCase):
  def test_fx_one(self):

    # Act
    root = newtons_method_scalar(scalar_function_one)

    # Assert
    self.assertAlmostEqual(0, scalar_function_one(root), 2)

  def test_fx_two(self):
    # Act
    root = newtons_method_scalar(scalar_function_two)

    # Assert
    self.assertAlmostEqual(0, scalar_function_two(root), 2)

  def test_x_y_one(self):
    # Arrange
    expected_root = -1

    # Act 
    root = newtons_method_scalar(scalar_function_one)

    # Assert
    self.assertAlmostEqual(0, abs(root - expected_root), 2)

  def test_x_y_two(self):
    # Arrange
    expected_root = 3

    # Act 
    root = newtons_method_scalar(scalar_function_two)

    # Assert
    self.assertAlmostEqual(0, abs(root - expected_root), 2)


if __name__ == '__main__':
    # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
    # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

# **Results**
All the alogrithms are implemented and the test are passing.

In [11]:
if __name__ == '__main__':
    # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
    # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.055s

OK


# **Discussion**
The results of the algorithms are sufficient to say that the algorithms are likely correctly implemented. There is still room for some further improvements in shape of guards that prevents from users of the methods from using them wrongly, such as type checks in earliest parts of the function. Also even if the types in the arguments are correct, there could be illgal states of an object (if the argument is an object), such as the numpy array could have illegal dimensions to perform a matrix multiplication.